In [77]:
import pandas as pd 
import os
import pyspark
from pyspark.sql import SparkSession
import docx
from docx import Document
import re
import csv 
import copy
import docx2txt
import sys

In [78]:
current_dir = os.getcwd()
raw_files = str(current_dir) + r"\raw_files"


# Each word document collected contains about 75 job description div containers, below is how one looks

In [18]:
list_of_files = os.listdir(raw_files)

my_text = docx2txt.process(raw_files+ "//" + list_of_files[0])
print(type(my_text))
#print(my_text)

separate_job_objects = my_text.split('\n\n--------------------')
print(len(separate_job_objects))
separate_job_objects[0]
#document_to_string = read_word(raw_files+ "//" + list_of_files[0])

<class 'str'>
76


'data analyst_dc_2024-03-22_page1\n\n\n\n\n\n<div id="viewJobSSRRoot"><div id="mosaic-aboveViewjobNav" class="mosaic-zone"></div><div class="fastviewjob jobsearch-ViewJobLayout--standalone css-10576t8 eu4oa1w0 hydrated" role="main"><div class="css-amnpyw e37uo190"></div><div class="css-8ua0kf eu4oa1w0"><div class="css-1xwak0u eu4oa1w0"><div class="css-jr3hje eu4oa1w0"><form action="/jobs" method="get" class="css-z48huh e37uo190"><span class=" css-1fr7b65 e6fjgti1"><label id="text-input-what-label" for="text-input-what" aria-hidden="true" class=" css-1ndt6qv eu4oa1w0">What</label><input type="text" aria-invalid="false" id="text-input-what" aria-label="what: job title, keywords, or company" name="q" autocomplete="off" placeholder="Job title, keywords, or company" value="" class="css-33odco e1jgz0i3"></span><span class=" css-1lglf0j e6fjgti1"><label id="text-input-where-label" for="text-input-where" aria-hidden="true" class=" css-1ndt6qv eu4oa1w0">Where</label><input type="text" aria-inva

# We want to extract the job description portion and clean up all remaining html tags and grammar

In [71]:
def read_word(path: str) -> str:
    ''' Creates a string object from a word document of job descriptions div containers
    
        Args: 
        path (str): file path to word document locaiton 
        
        Returns: 
        doc (str): a string made of the text of the word document 
       
    '''
    document = Document(path)
    
    # Initialize an empty string to hold the text block
    text_block = ""

    # Iterate through each paragraph in the document
    for para in document.paragraphs:
    # Append each paragraph's text to the text block, followed by a newline character
        text_block += para.text + "\n"
    
    return text_block 
    
def clean_job_description_paragraph(job_desc_list: list):
    ''' Take dirty job description html code already separated using ('-------'), and cleans html formatting using regex expressions
    
        Args: 
        job_desc_list (list): dirty job description html code 
        
        Returns: 
        job_description_string (str): string object of cleaned job description text
        
       
    '''
    
    job_description_string = ''
    
    phone_number_pattern = r'\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b'
    date_pattern = r'\(?\b\d{4}\)?[-.\s]?\d{2}[-.\s]?\d{2}\b'
    remove_top_html_pattern = r'</div><div id="jobDescriptionText"'
    remove_bottom_html_pattern = r'</div></div>'
    merged_tag_pattern = r'<(\w{1,2})(\w+)'
    replacement = r'\2'
    first_line_pattern = re.compile(r' class="[^"]*">\s')
    
    tags_to_remove = ['<div>', '</div>', '<p>', '</p>', '<br>', '</br>', '<ul>', '</ul>', '<i>', '</i>', '<b>', '</b>', '<li>', '</li>', '\n', '\n+', '<i>', "'", '<h4>', 
                        '</h4>', '</h3>', '<h3>', '<h2>', '</h2>', "’", r'/', r'\.00\b','  +']
    
    
    for job_desc_html in job_desc_list:
        
        try:
            job_desc_html_v2 = job_desc_html.split(remove_top_html_pattern)[1]
            job_desc_html_v3 = job_desc_html_v2.split(remove_bottom_html_pattern)[0]
        except:
            continue
        


        for items in tags_to_remove: 
            job_desc_html_v3 = re.sub(items, ' ', job_desc_html_v3)

        
        for regex_fliters in [first_line_pattern, phone_number_pattern, date_pattern]:
            job_desc_html_v3 = re.sub(regex_fliters, '', job_desc_html_v3)

        
        refined_job_desc = re.sub(merged_tag_pattern, replacement, job_desc_html_v3)

        job_description_string += "".join("Job Description:  ") + refined_job_desc +  "\n\n -------------------------------------------------------------------------------------- \n\n "
        
    return job_description_string.lower()

def docx_to_csv(csv_file_path: str, input_string_list: list):
    ''' Takes the separated job description list and creates a dataframe to readablility in AWS Glue
    
        Args: 
        csv_file_path (str): file path to save dataframe using word_docx title rename 
        input_string_list (list): list of job descriptions split using ('------') separator
        
       
    '''
    job_dataframe = pd.DataFrame()
    job_dataframe['numb_description'] = range(1, len(input_string_list))
    job_dataframe['job_description'] = input_string_list[:-1]
    job_dataframe.to_csv(csv_file_path, index=False)
    


input_folder = str(current_dir) + r"\raw_files" 
docx_output_folder = str(current_dir) + r"\processed_word_docs"
cleansed_folder = str(current_dir) + r"\cleansed_files"

list_of_files = os.listdir(input_folder)


for word_docx in list_of_files: 
    
    document_to_string = read_word(raw_files + r"\\" +  word_docx)
    separate_job_objects = document_to_string.split('\n\n--------------------') # Separate the job from a divider marker my bot set
    
    repaired_doc = clean_job_description_paragraph(separate_job_objects)
    
    cleansed_page = Document()
    cleansed_page.add_paragraph(repaired_doc)
    
    
    new_file_path = os.path.join(docx_output_folder, word_docx)
    cleansed_page.save(new_file_path)
    
    
    csv_split_file = repaired_doc.split('--------------------------------------------------------------------------------------')
    docx_to_csv(cleansed_folder + r"/" + word_docx[:-5] + '.csv', csv_split_file)

    


In [55]:
test = pd.read_csv(str(current_dir) + r"\cleansed_files\data-analyst_dmv_2024-05-05_page1.csv")
test.head()

def read_csv(csv_file_path):
    # Increase the maximum field size allowed
    csv.field_size_limit(sys.maxsize)

    with open(csv_file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            print(row)  # This should now handle larger fields without error
            
read_csv((str(current_dir) + r"\cleansed_files\data-analyst_dmv_2024-05-05_page1.csv"))

OverflowError: Python int too large to convert to C long

# This Cleansed Layer will not take the cleaned up job description paragraphs and use dictionaries to extract word frequencies

In [11]:
def create_dictionary_from_csv(path_to_csv: str) -> dict:
    ''' Creates environment folders and partitioning folders when new job title is created 
    
        Args: 
        path_to_csv (str): string path to csv file needed to make a frequency dictionary
        
        Returns:
        freq_dictionary (dict): blank frequency dictionary '''
    
    freq_dictionary = {} 

    with open(path_to_csv, mode='r', newline='') as file: 
        reader = csv.reader(file)
        next(reader, None) # Skips header
        for row in reader: 
            if row:
                freq_dictionary[row[0]] = 0
    
    return freq_dictionary

def create_cleansed_enviornment(list_of_files_in_raw: list, output_folder: str) -> list:
    ''' Creates environment folders and partitioning folders when new job title is created 
    
        Args: 
        list_of_files_in_raw (list): list of all files collected using os.listdir
        
        Returns:
        all_job_names (list): list of all unique names collected from raw files listed'''
        
    partition_folders = ['education', 'programming_languages', 'personality_traits', 'skillset', 'software', 'security_clearance', 'experience', 'salary']
    all_job_names = []
    
    for word_docx_filename in list_of_files_in_raw:

        position_title = word_docx_filename.split('_')[0]
    
        if position_title not in all_job_names: all_job_names.append(position_title)
            

    for items in partition_folders:
        if not os.path.exists(output_folder + "//" + items):
            os.makedirs(output_folder + "//" + items)
        

        
    return all_job_names

def extract_salary_from_paragraph(job_id: str, output_folder: str, word_docx_filename: str, document_to_string: str):
    ''' Cleans salary tuple in order to convert it into two dataframe columns after
        
        Args: 
        salary_tuple: tuple of low end, high end, and hourly/salary pay grades 
        
        Returns: 
        Tuple: a tuple of job low/high end pay ranges '''
    file_name_split = word_docx_filename.split('_')
    state = file_name_split[1]
    report_year = file_name_split[2].split('-')[0]
    
    salary_tuple = re.findall(r'(\$?\d{1,3}(?:k|,\d{1,3}|\d{1,3}))\s*?(?:to|-)\s*?(\$?\d{1,3}(?:k|,\d{1,3}|\d{1,3}))(?:\s*(?:per\s+|a\s+)?(hour|annually|year|yearly))?', document_to_string)
    pattern = r'[^a-zA-Z0-9\s]' # remove special characters
    low_end_pay = [] 
    high_end_pay = [] 
    
    
    for matches in salary_tuple: 
        low = re.sub(pattern, '', matches[0]) # remove special characters
        high = re.sub(pattern, '', matches[1]) # remove special characters
        
        low = re.sub('k', '000', low) # replace k with 000
        high = re.sub('k', '000', high) 
        
        
        if int(low) < 20000 or int(high) < 20000:
            
            if matches[2] == 'year' or matches[2] == 'annually':
                
                if len(low) < 5: low = int(low) * 1000
                if len(high) < 5: high = int(high) * 1000
                
                if int(low) < 20000 or int(high) < 20000:
                    continue
                    
            elif matches[2] == 'hour' or matches[2] == 'hourly':
                low = int(low) * 40 * 52 # Convert to salary
                high = int(high) * 40 * 52 # Convert to salary
                
            else: 
                continue
        
        low_end_pay.append(int(low))
        high_end_pay.append(int(high))
        
    salary_dataframe = pd.DataFrame()
    salary_dataframe['pay_low_end'] = low_end_pay
    salary_dataframe['pay_high_end'] = high_end_pay
    salary_dataframe['job_id'] = job_id
    salary_dataframe['state'] = state
    salary_dataframe['report_year'] = report_year
    
    salary_dataframe.to_csv(output_folder + "\\" + 'salary' + "\\"  + word_docx_filename + '.csv', index=False)

def extract_experience_from_paragraph(job_id: str, output_folder: str, word_docx_filename: str, document_to_string: str):
    """Retrieve or assign a unique job ID based on the job title.
    
    Args:
    list_of_experience: a list of all experience years collected from job descriptions
    
    Returns:
    years_experience: a cleaned list of professional experience
    """
    file_name_split = word_docx_filename.split('_')
    state = file_name_split[1]
    report_year = file_name_split[2].split('-')[0]
    
    years_experience = []
    list_of_experience = re.findall(re.compile(r'(\d+\+?|\d+\s*[-–to]\s*\d+)\s*(years?)'), document_to_string)
    
    for items in list_of_experience:
        years = re.sub(r'[^a-zA-Z0-9-]', '', items[0])
        
        if '-' in years: 
            years_array = years.split('-') 
            ranged_item = list(range(int(years_array[0]), int(years_array[1]) + 1))
            years_experience.extend(ranged_item)
        elif int(years) > 15: 
            continue
        else: years_experience.append(int(years))
    
    experience_db = pd.DataFrame()
    experience_db['years_experience_recorded'] = years_experience
    experience_db['job_id'] = job_id
    experience_db['state'] = state
    experience_db['report_year'] = report_year
    
    experience_db.to_csv(output_folder + "\\" + 'experience' + "\\"  + word_docx_filename + '.csv', index=False)

def get_job_id(job_name: str, file_path: str) -> int:
    """Retrieve or assign a unique job ID based on the job title.

    Args:
        job_name (str): The name of the job to retrieve or create an ID for.
        file_path (str): Path to the CSV file containing job categories.

    Returns:
        int: The job ID.
    """
    job_id_db = pd.read_csv(file_path)
    
    if not (job_id_db['job_title'].eq(job_name)).any():  # If the job title doesn't exist in job_categories.csv
        new_job_id = len(job_id_db)  # Assign the next ID
        new_data = pd.DataFrame({'job_id': [new_job_id], 'job_title': [job_name]})
        job_id_db = pd.concat([job_id_db, new_data], ignore_index=True)
        job_id_db.to_csv(file_path, index=False)  # Save the updated list
        return new_job_id
    else:
        # Return existing ID for the job title
        return job_id_db.loc[job_id_db['job_title'] == job_name, 'job_id'].iloc[0]
       
def write_dictionary_to_cleansed_layer(measurement_dictionary: dict, job_id: str, output_folder: str, folder: str, word_docx_filename: str): 
    """Write creates a dataframe object from dictionary passed, and writes it into a csv with the same file name
    
    Args:
    measurement_dictionary (dict): frequency dictionary.
    job_id (int): unique job id.
    output_folder (str): path to cleansed layer.
    folder (str): job specific folder in cleansed layer.
    word_docx_filename (str): name of file being used
    
    """
    
    file_name_split = word_docx_filename.split('_')
    state = file_name_split[1]
    report_year = file_name_split[2].split('-')[0]
    
    for name, dictionary in measurement_dictionary.items():
        
        dataframe = pd.DataFrame(list(measurement_dictionary.items()), columns = [folder, 'frequency'])
        dataframe['job_id'] = job_id  # Adding job_id column
        dataframe['state'] = state  # Adding state column
        dataframe['report_year'] = report_year  # Adding report year column
        dataframe.to_csv(output_folder + "\\" + folder + "\\"  + word_docx_filename + '.csv', index=False)
        
def mark_phrases(word_doc_text: str, words_for_marking_desc: list) -> str:
    ''' Function takes multiple worded phrases from dictionaries and replaced space with '-' in order to mark them before counting 
        
        Args: 
        word_doc_text (str): word documented converted into a string 
        words_for_marking (list) : list of phrases that will be marked to count accurately 
        
        Returns: 
        word_doc_text (str): word document with marked text '''
    for phrase in words_for_marking_desc:

        word_doc_text = re.sub(phrase, phrase.replace(' ', '-'), word_doc_text)
        
    return word_doc_text

def find_special_characters(s: str):
    '''
    Function removes non-alphanumeric characters
    
    Args: 
    s (str): text based value 
    
    Returns: 
    s (str): cleaned text 
    
    '''
    # This pattern matches any character that is not a letter or a number
    pattern = re.compile('[^a-zA-Z0-9]')
    # Find all non-alphanumeric characters in the string
    special_chars = pattern.findall(s)
    unique_special_chars = set(special_chars)
    
    for special_character in unique_special_chars: 
        s = s.replace(special_character, "\\" + special_character)
    return s

def create_key_glossary_from_dict_shells(dictionary_skeletons_folder: str):
    # Specify the path to the file
    glossary_path = dictionary_skeletons_folder + '\dict_key_glossary.csv'
    
    
    # Check if the file exists
    if os.path.exists(glossary_path):
        glossary_db = pd.read_csv(glossary_path)
    else:
        glossary_db = pd.DataFrame(columns=['keys'])
        
        for file in os.listdir(dictionary_skeletons_folder):
            if file.endswith('.csv'):
                keyword_list = pd.read_csv(dictionary_skeletons_folder + r"\\" +  file).iloc[:, 0].tolist()
                new_df = pd.DataFrame(keyword_list, columns=['keys'])
                glossary_db = pd.concat([glossary_db, new_df], ignore_index=True)
        
        glossary_db.to_csv(glossary_path, index=False)
        
    list_of_hypend_words = [x for x in glossary_db.iloc[:, 0].tolist() if "-" in x]
    words_without_hyphens = list(map(lambda word: word.replace('-', ' '), list_of_hypend_words))
    words_without_hyphens_desc = sorted(words_without_hyphens, key=len, reverse=True)
    
    return words_without_hyphens_desc



In [13]:
import os
import copy


input_folder = str(current_dir) + r"\cleansed_files"
output_folder = str(current_dir) + r"\curated_files"
job_category_folder = str(current_dir) + r"\artifacts\dependencies\job_categories.csv"
dictionary_skeletons_folder = str(current_dir) + r"\artifacts\dictionary_shells"

# Pre-load dictionary shells
dictionary_shells = {
    "education": create_dictionary_from_csv(os.path.join(dictionary_skeletons_folder, "education.csv")),
    "programming_languages": create_dictionary_from_csv(os.path.join(dictionary_skeletons_folder, "programming_languages.csv")),
    "personality_traits": create_dictionary_from_csv(os.path.join(dictionary_skeletons_folder, "personality_traits.csv")),
    "skills": create_dictionary_from_csv(os.path.join(dictionary_skeletons_folder, "skills.csv")),
    "software": create_dictionary_from_csv(os.path.join(dictionary_skeletons_folder, "software.csv")),
    "security_clearance": create_dictionary_from_csv(os.path.join(dictionary_skeletons_folder, "security_clearance.csv"))
}

words_for_marking_desc = create_key_glossary_from_dict_shells(dictionary_skeletons_folder)
list_of_files = os.listdir(input_folder)
unique_job_names = create_cleansed_enviornment(list_of_files, output_folder)

for job_titles in unique_job_names:
    
    job_id = get_job_id(job_titles, job_category_folder)
    
    job_specific_files = [x for x in list_of_files if job_titles in x]
    
    for word_docx_filename in job_specific_files:
        print(word_docx_filename)
        
        document_to_string = read_word(os.path.join(input_folder, word_docx_filename))
        
        document_to_string_v2 = mark_phrases(document_to_string, words_for_marking_desc)

        extract_experience_from_paragraph(job_id, output_folder, word_docx_filename, document_to_string_v2)
        extract_salary_from_paragraph(job_id, output_folder, word_docx_filename, document_to_string_v2)

        # Reset dictionaries to initial template by deep copying
        dictionaries = {k: copy.deepcopy(v) for k, v in dictionary_shells.items()}
        
        partition_folders = ['education', 'programming_languages', 'personality_traits', 'skillset', 'software', 'security_clearance']

        for counter, (category, dictionary) in enumerate(dictionaries.items()):
            for key in dictionary:
                regex_key = find_special_characters(key)
                pattern = r"(?<=\s)[\.,(]*" + regex_key + r"[\.,)]*(?=\s)"
                dictionary[key] = len(re.findall(pattern, document_to_string_v2))
            write_dictionary_to_cleansed_layer(dictionary, job_id, output_folder, partition_folders[counter], word_docx_filename[:-5])
            counter += 1


data-analyst_dmv_2024-05-05_page1.docx
data-analyst_dmv_2024-05-05_page10.docx
data-analyst_dmv_2024-05-05_page11.docx
data-analyst_dmv_2024-05-05_page12.docx
data-analyst_dmv_2024-05-05_page2.docx
data-analyst_dmv_2024-05-05_page3.docx
data-analyst_dmv_2024-05-05_page4.docx
data-analyst_dmv_2024-05-05_page5.docx
data-analyst_dmv_2024-05-05_page6.docx
data-analyst_dmv_2024-05-05_page7.docx
data-analyst_dmv_2024-05-05_page8.docx
data-analyst_dmv_2024-05-05_page9.docx
software-engineer_dmv_2024-05-05_page1.docx
software-engineer_dmv_2024-05-05_page10.docx
software-engineer_dmv_2024-05-05_page11.docx
software-engineer_dmv_2024-05-05_page12.docx
software-engineer_dmv_2024-05-05_page13.docx
software-engineer_dmv_2024-05-05_page14.docx
software-engineer_dmv_2024-05-05_page15.docx
software-engineer_dmv_2024-05-05_page16.docx
software-engineer_dmv_2024-05-05_page17.docx
software-engineer_dmv_2024-05-05_page18.docx
software-engineer_dmv_2024-05-05_page19.docx
software-engineer_dmv_2024-05-05_pa

# Reporting Layer

In [231]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('headstart_pipeline_code') \
                    .getOrCreate()

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
spark.conf.set("spark.sql.execution.arrow.enabled","true")

# Education Reporting Layer

In [232]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"
skeletons_mapping_folder = str(current_dir) + r"\artifacts\category_mapping"


education = spark.read.option("header", True).csv('education_test.csv')
education_map = spark.read.option("header", True).csv(skeletons_mapping_folder + r"\education_category_mapping.csv")

education.createOrReplaceTempView("education_database") 
education_map.createOrReplaceTempView("edu_map") 

mapped_database = spark.sql('''
        SELECT ed.job_id, 
               em.degree_category_id, 
               SUM(ed.frequency) AS frequency, 
               ed.state, 
               ed.report_year
        FROM education_database ed
        LEFT JOIN edu_map em 
            ON ed.education = em.degree 
        GROUP BY em.degree_category_id, ed.job_id, ed.state, ed.report_year
        ORDER BY ed.job_id ASC, em.degree_category_id, report_year DESC
    ''')


mapped_database.show()

+------+------------------+---------+-----+-----------+
|job_id|degree_category_id|frequency|state|report_year|
+------+------------------+---------+-----+-----------+
|     0| associates degree|      1.0|  dmv|       2024|
|     0|  bachelors degree|     48.0|  dmv|       2024|
|     0|        highschool|      0.0|  dmv|       2024|
|     0|    masters degree|      9.0|  dmv|       2024|
|     0|               phd|      3.0|  dmv|       2024|
+------+------------------+---------+-----+-----------+



# Job Description Experience In Years For Positions Reporting Layer

In [233]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"

experience_db = spark.read.option("header", True).csv('experience_test.csv')

experience_db.createOrReplaceTempView('experience_database') 

result_db = spark.sql(''' SELECT job_id, 
                                 CAST(years_experience_recorded AS INT) AS years_experience, 
                                 COUNT(years_experience_recorded) AS frequency,
                                 state, 
                                 report_year
                          FROM experience_database 
                          GROUP BY years_experience_recorded, job_id, state, report_year
                          ORDER BY job_id ASC, years_experience ASC, report_year DESC
    ''')

result_db.show()

+------+----------------+---------+-----+-----------+
|job_id|years_experience|frequency|state|report_year|
+------+----------------+---------+-----+-----------+
|     0|               0|        1|  dmv|       2024|
|     0|               1|        5|  dmv|       2024|
|     0|               2|       16|  dmv|       2024|
|     0|               3|       22|  dmv|       2024|
|     0|               4|       19|  dmv|       2024|
|     0|               5|       18|  dmv|       2024|
|     0|               6|        6|  dmv|       2024|
|     0|               7|        4|  dmv|       2024|
|     0|               8|        8|  dmv|       2024|
|     0|               9|        4|  dmv|       2024|
|     0|              10|        9|  dmv|       2024|
|     0|              11|        3|  dmv|       2024|
|     0|              12|        1|  dmv|       2024|
|     0|              14|        2|  dmv|       2024|
+------+----------------+---------+-----+-----------+



# Job Description Personality Trait Frequencies For Positions Reporting Layer

In [234]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"

personality_db = spark.read.option("header", True).csv('personality_test.csv')

personality_db.createOrReplaceTempView("personality_db")

result_df = spark.sql("""
                        SELECT job_id, 
                               personality_traits, 
                               SUM(frequency), 
                               state, 
                               report_year
                        FROM personality_db 
                        GROUP BY personality_traits, job_id, state, report_year
                        ORDER BY personality_traits ASC, job_id ASC, report_year DESC
                        
                      """)

result_df.show()

+------+------------------+--------------+-----+-----------+
|job_id|personality_traits|sum(frequency)|state|report_year|
+------+------------------+--------------+-----+-----------+
|     0|          abrasive|           5.0|  dmv|       2024|
|     0|          abrasive|          10.0|  dmv|       2023|
|     1|          abrasive|           0.0|  dmv|       2024|
|     0|            abrupt|           0.0|  dmv|       2024|
|     1|            abrupt|           0.0|  dmv|       2024|
|     0|      absentminded|           0.0|  dmv|       2024|
|     1|      absentminded|           0.0|  dmv|       2024|
|     0|        accessible|           2.0|  dmv|       2024|
|     1|        accessible|           0.0|  dmv|       2024|
|     0|            active|          22.0|  dmv|       2024|
|     1|            active|           0.0|  dmv|       2024|
|     0|         adaptable|           1.0|  dmv|       2024|
|     1|         adaptable|           0.0|  dmv|       2024|
|     0|         admirab

# Programming Language Reporting Layer

In [235]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"

programming_db = spark.read.option("header", True).csv('programming_test.csv')
programming_db.createOrReplaceTempView("programming_db")

result_df = spark.sql(""" 
                         SELECT job_id, 
                                programming_languages,
                                SUM(frequency) AS frequency, 
                                state, 
                                report_year
                         FROM programming_db
                         GROUP BY programming_languages, job_id, state, report_year
                         ORDER BY programming_languages ASC, job_id ASC, report_year DESC
                         
                      """)
result_df.show()

+------+---------------------+---------+-----+-----------+
|job_id|programming_languages|frequency|state|report_year|
+------+---------------------+---------+-----+-----------+
|     0|                 abap|      0.0|  dmv|       2024|
|     0|         actionscript|      0.0|  dmv|       2024|
|     0|                  ada|      2.0|  dmv|       2024|
|     0|          angelscript|      0.0|  dmv|       2024|
|     0|                 apex|      0.0|  dmv|       2024|
|     0|                  apl|      0.0|  dmv|       2024|
|     0|          applescript|      0.0|  dmv|       2024|
|     0|    assembly-language|      0.0|  dmv|       2024|
|     0|                  awk|      0.0|  dmv|       2024|
|     0|            ballerina|      0.0|  dmv|       2024|
|     0|                bison|      0.0|  dmv|       2024|
|     0|                   c#|      0.0|  dmv|       2024|
|     0|                  c++|      1.0|  dmv|       2024|
|     0|               ceylon|      0.0|  dmv|       202

# Salary Data Reporting Layer Code 

In [236]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"

salary_db = spark.read.option("header", True).option("inferSchema", True).csv('salary_test.csv')
salary_db.printSchema()

salary_db.createOrReplaceTempView("salary_db")

result_df = spark.sql("""
                        SELECT job_id, 
                               pay_low_end AS low_end_salary, 
                               pay_high_end AS high_end_salary, 
                               state, 
                               report_year
                        FROM salary_db 
                        ORDER BY pay_low_end ASC
                      """
                     )
result_df.show()

root
 |-- pay_low_end: integer (nullable = true)
 |-- pay_high_end: integer (nullable = true)
 |-- job_id: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- report_year: integer (nullable = true)

+------+--------------+---------------+-----+-----------+
|job_id|low_end_salary|high_end_salary|state|report_year|
+------+--------------+---------------+-----+-----------+
|     0|         39000|          59007|  dmv|       2024|
|     0|         45400|          93000|  dmv|       2024|
|     0|         45400|          93000|  dmv|       2024|
|     0|         52100|         119000|  dmv|       2024|
|     0|         52100|         119000|  dmv|       2024|
|     0|         57500|         117900|  dmv|       2024|
|     0|         68000|          78000|  dmv|       2024|
|     0|         72000|         155000|  dmv|       2024|
|     0|         72000|         155000|  dmv|       2024|
|     0|         73100|         166000|  dmv|       2024|
|     0|         75570|        

# Security Clearance Reporting Layer Code 

In [237]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"
skeletons_mapping_folder = str(current_dir) + r"\artifacts\category_mapping"


security_clearance_db = spark.read.option("header", True).option("inferSchema", True).csv('security_clearance_test.csv')
security_clearance_mapping = spark.read.option("header", True).csv(skeletons_mapping_folder + r"\security_clearance_mapping.csv")

security_clearance_db.createOrReplaceTempView("security_clearance_db")
security_clearance_mapping.createOrReplaceTempView("security_clearance_mapping_db")

result_df = spark.sql("""
                        
                        SELECT scdb.job_id,
                               map.clearance_category, 
                               SUM(scdb.frequency) AS frequency,
                               scdb.state,
                               scdb.report_year
                               FROM security_clearance_db scdb
                                   LEFT JOIN security_clearance_mapping_db map 
                                   ON scdb.security_clearance = map.security_clearance
                               GROUP BY map.clearance_category, scdb.job_id, scdb.state, scdb.report_year
                               ORDER BY map.clearance_category ASC, scdb.job_id ASC, scdb.report_year DESC
                      """
                     )
result_df.show()

+------+--------------------+---------+-----+-----------+
|job_id|  clearance_category|frequency|state|report_year|
+------+--------------------+---------+-----+-----------+
|     0|        public trust|        7|  dmv|       2024|
|     0|        public trust|        6|  dmv|       2023|
|     1|        public trust|        5|  dmv|       2024|
|     2|        public trust|        4|  dmv|       2023|
|     0|    secret clearance|       27|  dmv|       2024|
|     0|special access pr...|        0|  dmv|       2024|
|     0|top secret clearance|       21|  dmv|       2024|
+------+--------------------+---------+-----+-----------+



# Skillset Reporting Layer Code 

In [249]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"
skeletons_mapping_folder = str(current_dir) + r"\artifacts\category_mapping"


skillset_db = spark.read.option("header", True).option("inferSchema", True).csv('skillset_test.csv')
skills_category_map = spark.read.option("header", True).csv(skeletons_mapping_folder + r"\skills_category_mapping.csv")

skillset_db.createOrReplaceTempView("skillset_db") 
skills_category_map.createOrReplaceTempView("skills_category_map") 

result_df = spark.sql(""" 
                        SELECT skills.job_id,
                               skills_map.skill_category,
                               skills.skillset,
                               SUM(skills.frequency) AS frequency, 
                               skills.state, 
                               skills.report_year
                        FROM skillset_db skills
                            LEFT JOIN skills_category_map skills_map
                            ON skills.skillset = skills_map.skillset
                        GROUP BY skills.skillset, skills.job_id, skills_map.skill_category, skills.state, skills.report_year
                        ORDER BY skills_map.skill_category ASC, skills.skillset ASC, skills.job_id ASC, skills.report_year DESC
                      """
                     )

result_df.show(20)

+------+-----------------+--------------------+---------+-----+-----------+
|job_id|   skill_category|            skillset|frequency|state|report_year|
+------+-----------------+--------------------+---------+-----+-----------+
|     0|management skills|    active-listening|        0|  dmv|       2024|
|     0|management skills|        adaptability|        0|  dmv|       2024|
|     0|management skills|       assertiveness|        0|  dmv|       2024|
|     0|management skills| attention-to-detail|       15|  dmv|       2024|
|     0|management skills|cloud-storage-and...|        0|  dmv|       2024|
|     0|management skills|       collaboration|       16|  dmv|       2024|
|     0|management skills|          compassion|        0|  dmv|       2024|
|     0|management skills|          confidence|        2|  dmv|       2024|
|     0|management skills|          creativity|        3|  dmv|       2024|
|     0|management skills|critical-observation|        0|  dmv|       2024|
|     0|mana

# Software Reporting Layer Code

In [281]:
s3_input = str(current_dir) + r"\curated_files"
s3_output = str(current_dir) + r"\reporting_layer"
skeletons_mapping_folder = str(current_dir) + r"\artifacts\category_mapping"


software_db = spark.read.option("header", True).option("inferSchema", True).csv('software_test.csv')
software_map = spark.read.option("header", True).csv(skeletons_mapping_folder + r"\software_category_mapping.csv")


software_db.createOrReplaceTempView("software_db") 
software_map.createOrReplaceTempView("software_map") 
    
result_df = spark.sql("""
                        SELECT sdb.job_id, 
                               sdb.software,
                               map.software_category, 
                               SUM(sdb.frequency) AS frequency, 
                               sdb.state, 
                               sdb.report_year 
                        FROM software_db sdb
                            LEFT JOIN software_map map 
                            ON sdb.software = map.software_name
                        GROUP BY sdb.software, sdb.job_id, map.software_category, sdb.state, sdb.report_year
                        ORDER BY map.software_category ASC, sdb.software ASC, sdb.job_id ASC, sdb.report_year ASC
                      """
                     )

result_df.show()

+------+--------------------+-----------------+---------+-----+-----------+
|job_id|            software|software_category|frequency|state|report_year|
+------+--------------------+-----------------+---------+-----+-----------+
|     0|      autodesk-revit|        3d design|        0|  dmv|       2024|
|     0|            sketchup|        3d design|        0|  dmv|       2024|
|     0|       alibaba-cloud|  cloud platforms|        0|  dmv|       2024|
|     0|                 aws|  cloud platforms|        7|  dmv|       2023|
|     0|                 aws|  cloud platforms|       10|  dmv|       2024|
|     1|                 aws|  cloud platforms|        8|  dmv|       2023|
|     1|                 aws|  cloud platforms|        9|  dmv|       2024|
|     0|               azure|  cloud platforms|        3|  dmv|       2024|
|     0|azure-virtual-mac...|  cloud platforms|        0|  dmv|       2024|
|     0|       cloud-foundry|  cloud platforms|        0|  dmv|       2024|
|     0|    

In [ ]:
import re

# Define regex patterns as constants
PHONE_NUMBER_PATTERN = re.compile(r'\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
DATE_PATTERN = re.compile(r'\(?\b\d{4}\)?[-.\s]?\d{2}[-.\s]?\d{2}\b')
REMOVE_TOP_HTML_PATTERN = re.compile(r'</div><div id="jobDescriptionText"')
REMOVE_BOTTOM_HTML_PATTERN = re.compile(r'</div></div>')
MERGED_TAG_PATTERN = re.compile(r'<(\w{1,2})(\w+)')
FIRST_LINE_PATTERN = re.compile(r' class="[^"]*">\s')

# Define tags and characters to remove
TAGS_TO_REMOVE = re.compile(r'|'.join([
    '<div>', '</div>', '<p>', '</p>', '<br>', '</br>', '<ul>', '</ul>',
    '<i>', '</i>', '<b>', '</b>', '<li>', '</li>', '\n', '<i>', "'", 
    '<h4>', '</h4>', '<h3>', '</h3>', '<h2>', '</h2>', "’", r'/', r'\.00\b', '  +'
]))

def clean_job_description_paragraph(job_desc_list):
    job_description_string = []

    for job_desc_html in job_desc_list:
        try:
            # Extract the relevant part of the HTML
            job_desc_html_part = REMOVE_TOP_HTML_PATTERN.split(job_desc_html)[1].split(REMOVE_BOTTOM_HTML_PATTERN.pattern)[0]
        except IndexError:
            continue

        # Remove specified tags and characters
        job_desc_html_part = TAGS_TO_REMOVE.sub(' ', job_desc_html_part)

        # Apply regex filters
        for regex_filter in [FIRST_LINE_PATTERN, PHONE_NUMBER_PATTERN, DATE_PATTERN]:
            job_desc_html_part = regex_filter.sub('', job_desc_html_part)

        # Refine the job description
        refined_job_desc = MERGED_TAG_PATTERN.sub(r'\2', job_desc_html_part)

        # Append to the result list with a formatted header
        job_description_string.append("\u0332".join("Job Description:  ") + refined_job_desc + "\n\n -------------------------------------------------------------------------------------- \n\n ")

    return ''.join(job_description_string).lower()


In [ ]:
        for job_name in all_job_names: 
            job_name_prefix = prefix + job_name + "/"
            job_name_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=job_name_prefix)
            
            if 'Contents' in job_name_response:
                # Folder exists
                logger.info(job_name + " folder exists and code will begin executing")
            else:
                logger.info(job_name + " folder not found, will add it now")   
                s3.put_object(Bucket=bucket_name, Key=job_name_prefix)